# Perform regression on parameterized surfaces

## Imports

In [1]:
# imports

import trimesh
import os
import sys
import subprocess
import torch
import numpy as np
os.environ["GEOMSTATS_BACKEND"] = "pytorch"
import geomstats.backend as gs
import my28brains.discrete_surfaces

INFO: Using pytorch backend


In [2]:
gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
work_dir = os.getcwd()

my28brains_dir = os.path.join(work_dir, "my28brains")

sys_dir = os.path.dirname(work_dir)
sys.path.append(sys_dir)
sys.path.append(my28brains_dir)

# this forces the notebook to re-load
import importlib
importlib.reload(my28brains.discrete_surfaces)

from my28brains.discrete_surfaces import (
    DiscreteSurfaces,
    ElasticMetric,
)

import my28brains.default_config as default_config

## Generate parameterized geodesic from sphere to ellipse using `discrete_curves.py`.

In [5]:
#create sphere mesh
sphere = trimesh.creation.icosphere(subdivisions=3, radius=30.0)

#Create ellipsoid mesh
# Create a scaling matrix for the semi-axes lengths
scales = np.array([2, 2, 3])
scale_matrix = np.diag(scales)

scale_matrix = gs.array(scale_matrix)

# Apply the scaling transformation to the mesh vertices
scaled_vertices = sphere.vertices.dot(scale_matrix)

# Create a new mesh with the scaled vertices
ellipsoid = trimesh.Trimesh(vertices=scaled_vertices, faces=sphere.faces)

#Define Surface Space
SURFACE_SPACE = DiscreteSurfaces(faces = sphere.faces)

METRIC = ElasticMetric(
    space = SURFACE_SPACE,
    a0=default_config.a0,
    a1=default_config.a1,
    b1=default_config.b1,
    c1=default_config.c1,
    d1=default_config.d1,
    a2=default_config.a2,)

# CREATE GEODESIC
n_vertices = sphere.vertices.shape[0]
print(n_vertices)

def geodesics_sphere_to_ellipsoid(
    n_geodesics=1, n_times=5
):
    """Generate a dataset of geodesics that transform spheres into ellipsoids."""
    dim = 3
    
    geodesics = gs.zeros((n_geodesics, n_times, n_vertices, dim))
    times = gs.arange(0, 1, 1 / n_times)
    for i_geodesic in range(n_geodesics):
        start_surface = gs.array(sphere.vertices)
        end_surface = gs.array(ellipsoid.vertices)
        print(start_surface.shape)
        geodesic = METRIC.geodesic(initial_point = start_surface, end_point = end_surface)
        
        geodesics[i_geodesic] = geodesic

    return geodesics

geodesics = geodesics_sphere_to_ellipsoid()
geodesic = geodesics[0]

642
torch.Size([642, 3])
geodesic shapetorch.Size([3, 642, 3])
flattened geodesic shapetorch.Size([5778])
2D point arraytorch.Size([642, 3])
face_areas (642, 3)
shape of point in surface_one_forms: torch.Size([642, 3])
vertex_0: torch.Size([1280, 3])
surface_one_forms: torch.Size([1280, 3, 2])
surface_metric_matrices in DiscreteSurfaces: tensor([[[30.9589, 14.9811],
         [14.9811, 37.5415]],

        [[30.8048, 15.3590],
         [15.3590, 38.1298]],

        [[31.5015, 15.3814],
         [15.3814, 37.1381]],

        ...,

        [[41.5570, 27.4475],
         [27.4475, 54.8951]],

        [[42.3150, 27.3361],
         [27.3361, 53.6944]],

        [[54.8951, 27.4475],
         [27.4475, 42.3150]]])
face_areas determinant: tensor([30.6237, 30.6379, 30.5502,  ..., 39.0885, 39.0489, 39.6171])
2D point arraytorch.Size([642, 3])
face_areas (642, 3)
shape of point in surface_one_forms: torch.Size([642, 3])
vertex_0: torch.Size([1280, 3])
surface_one_forms: torch.Size([1280, 3, 2])
surf

 This problem is unconstrained.


# GEODESIC REGRESSION

In [4]:
# Regression imports

import matplotlib.pyplot as plt

import geomstats.backend as gs
import geomstats.visualization as visualization
from geomstats.geometry.special_euclidean import SpecialEuclidean
from geomstats.learning.frechet_mean import FrechetMean, variance
from geomstats.learning.geodesic_regression import GeodesicRegression

In [7]:
gr = GeodesicRegression(
        SURFACE_SPACE,
        metric=METRIC,
        center_X=False,
        method="riemannian",
        max_iter=100,
        init_step_size=0.1,
        verbose=True,
        initialization="frechet",
    )
gr.fit(geodesic, geodesic, compute_training_score=True)

intercept_hat, beta_hat = gr.intercept_, gr.coef_

AttributeError: 'DiscreteSurfaces' object has no attribute 'default_point_type'